참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengege Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 07.003<br>ex07.003

부정정보: 이중적분<br>Statically Indeterminate Beam : Double integration

p. 296

### 문제에서 주어진 변수<br>Given Parameters

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB_m, a_m = sy.symbols('L_AB_m, a_m', real=True, positive=True)

s_d = {
    L_AB_m: 4,
    a_m: 2,
}


#### 하중<br>Load

In [ ]:
P_N = sy.symbols('P_N', real=True)

s_d.update(
    {
        P_N: -5000,
    }
)


#### 잉여구속<br>Residual constraint

In [ ]:
delta_B_m = sy.symbols('delta_B_m', real=True)
s_d[delta_B_m] = 30e-3

#### 재료와 단면 특성<br>Material & section properties

In [ ]:
E_Pa, I_m4 = sy.symbols('E_Pa, I_m4', positive=True)

s_d.update(
    {
        E_Pa: 10e9,
        I_m4: 20e6 * (1e-3) ** 4,
    }
)


#### 자유물체도<br>Free body diagram

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_AB_m.subs(s_d), 'text':'B'},
]

reaction_list = [
    {'x_m': 0},
    {'x_m': L_AB_m.subs(s_d)},
]

dist_load_list = []

v_load_list = [
    {'x_m':a_m.subs(s_d), 'sign': -1},
    {'x_m':a_m.subs(s_d) * 1.1, 'sign': +1}
]

moment_list = [
    {'x_m': 0, 'direction': 'ccw', 'text': 'M', 'open':'right'},
]

draw_diagrams.draw_beam(L_AB_m.subs(s_d), points_list, reaction_list, v_load_list=v_load_list, dist_load_list=dist_load_list, moment_list=moment_list)

In [ ]:
x_m = sy.symbols('x_m', nonnegative=True)
x_A_m = 0
x_B_m = L_AB_m


### 반력<br>Reaction forces

In [ ]:
R_A_N, R_B_N, RM_A_Nm = sy.symbols('R_A_N, R_B_N, RM_A_Nm', real=True)

### 평형방정식<br>Equilibrium Equations

In [ ]:
fy_eq = sy.Eq(R_A_N + R_B_N + P_N)

In [ ]:
fy_eq

In [ ]:
m_eq = sy.Eq(RM_A_Nm + R_B_N * L_AB_m + P_N * a_m)

In [ ]:
m_eq

### 굽힘모멘트선도 <br>Bending moment diagram

In [ ]:
M = - RM_A_Nm + R_A_N * x_m + P_N * (x_m - a_m) * sy.Heaviside(x_m - a_m)

In [ ]:
M

### 처짐<br>Deflection

In [ ]:
EInu_Nm3 = sy.Function('EI\\nu_Nm3')
# http://docs.sympy.org/latest/modules/solvers/ode.html#dsolve
# http://docs.sympy.org/latest/tutorial/calculus.html
EInu_pp_Nm = sy.Derivative(EInu_Nm3(x_m), x_m, x_m)
defl_deq = sy.Eq(EInu_pp_Nm, M)

In [ ]:
defl_deq

In [ ]:
EInu_Nm3_sol_eq = sy.dsolve(defl_deq, EInu_Nm3(x_m))

In [ ]:
EInu_Nm3_sol = EInu_Nm3_sol_eq.rhs

In [ ]:
EInu_Nm3_sol